In [1]:
import os
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import warnings
import pandas as pd
import math

from networkx.algorithms import community

warnings.simplefilter('ignore')

In [2]:
class RegressionMetrics:

    """
    Metoda care calculeaza eroarea de regresie single-output pe baza unei functii
    @params: predicted - the values computed by the model
             target - the actual values
             function - the function to be applied to the difference between the values
    @return: float
    """
    @classmethod
    def __SingleError(cls, predicted, target, function = lambda x : x):
        return (1 / len(predicted)) * sum([function(p - t) for p, t in zip(predicted, target)])

    """
    Metoda care calculeaza Mean Absolute Error pentru regresie single-output 
    @params: predicted - the values computed by the model
             target - the actual values
    @return: float
    """
    @classmethod
    def SingleMAE(cls, predicted, target):
        return RegressionMetrics.__SingleError(predicted, target, lambda x : abs(x))

    """
    Metoda care calculeaza Mean Square Error pentru regresie single-output
    @params: predicted - the values computed by the model
             target - the actual values
    @return: float
    """
    @classmethod
    def SingleMSE(cls, predicted, target):
        return RegressionMetrics.__SingleError(predicted, target, lambda x : x ** 2)

    """
    Metoda care calculeaza Root Mean Square Error pentru regresie single-output 
    @params: predicted - the values computed by the model
             target - the actual values
    @return: float
    """
    @classmethod
    def SingleRMSE(cls, predicted, target):
        return math.sqrt(RegressionMetrics.SingleMSE(predicted, target))
    
    """
    Metoda care calculeaza eroarea de regresie multi-output pe baza unei functii
    @params: predicted - the values computed by the model
             target - the actual values
             function - the function to be applied to the difference between the values
    @return: float
    """
    @classmethod
    def __MultiError(cls, predicted, target, errorFunction):
        return sum([errorFunction(pre, tar) for pre, tar in zip([[pre[i] for pre in predicted] for i in range(len(predicted[0]))], [[tar[i] for tar in target] for i in range(len(target[0]))])])

    """
    Metoda care calculeaza Mean Absolute Error pentru regresie multi-output 
    @params: predicted - the values computed by the model
             target - the actual values
    @return: float
    """
    @classmethod 
    def MultiMAE(cls, predicted, target):
        return RegressionMetrics.__MultiError(predicted, target, RegressionMetrics.SingleMAE)

    """
    Metoda care calculeaza Mean Square Error pentru regresie multi-output 
    @params: predicted - the values computed by the model
             target - the actual values
    @return: float
    """
    @classmethod 
    def MultiMSE(cls, predicted, target):
        return RegressionMetrics.__MultiError(predicted, target, RegressionMetrics.SingleMSE)

    """
    Metoda care calculeaza Root Mean Square Error pentru regresie multi-output 
    @params: predicted - the values computed by the model
             target - the actual values
    @return: float
    """
    @classmethod
    def MultiRMSE(cls, predicted, target):
        return math.sqrt(RegressionMetrics.MultiMSE(predicted, target))

In [3]:
class ClassificationMetrics:

    """
    Metoda care calculeaza acuratetea de predictie pentru o clasificare single label
    @params: predicted - the classes predicted by the model
             target - the actual classes
    @return: integer
    """
    @classmethod
    def Accuracy(cls, predicted, target):
        return (1 / len(predicted)) * sum([p == t for p, t in zip(predicted, target)])
    
    """
    Metoda care calculeaza precizia pentru o clasificare binara
    @params: predicted - the classes predicted by the model
             target - the actual classes
    @return: integer
    """
    @classmethod
    def BinaryPrecision(cls, predicted, target, trueLabel = 1):
        return (sum([p == t for p, t in zip(predicted, target) if t == trueLabel])) / (sum([p == trueLabel for p in predicted]))
    
    """
    Metoda care calculeaza recall-ul pentru o clasificare binara
    @params: predicted - the classes predicted by the model
             target - the actual classes
    @return: integer
    """
    @classmethod
    def BinaryRecall(cls, predicted, target, trueLabel = 1):
        return sum([p == trueLabel for p in predicted]) / sum([t == trueLabel for t in target])

    """
    Metoda care calculeaza acuratetea, precizia si recall-ul pentru o clasificare binara
    @params: predicted - the classes predicted by the model
             target - the actual classes
    @return: un dictionar care cuprinde metricile
    """
    @classmethod
    def BinaryMetrics(cls, predicted, target, trueLabel, falseLabel):
        accuracy = sum([label == targetLabel for label, targetLabel in zip(predicted, target)]) / len(predicted)

        TP = sum([plabel == trueLabel for plabel, tlabel in zip(predicted, target) if tlabel == trueLabel])
        TN = sum([plabel == falseLabel for plabel, tlabel in zip(predicted, target) if tlabel == falseLabel])
        FP = sum([plabel == trueLabel for plabel, tlabel in zip(predicted, target) if tlabel == falseLabel])
        FN = sum([plabel == falseLabel for plabel, tlabel in zip(predicted, target) if tlabel == trueLabel])

        precisionPos = TP / (TP + FP)
        recallPos = TP / (TP + FN)
        precisionNeg = TN / (TN + FN)
        recallNeg = TN / (TN + FP)

        return {'accuracy': accuracy, 'precision positive': precisionPos, 'recall positive': recallPos, 'precision negative': precisionNeg, 'recall negative': recallNeg}
    
    """
    Metoda care calculeaza precizia de predictie pentru o clasificare multi class
    @params: predicted - the classes predicted by the model
             target - the actual classes
    @return: integer
    """
    @classmethod
    def MultiPrecision(cls, predicted, target):
        return dict([x, sum([y == z for y, z in zip(predicted, target) if y == x]) / sum(y == x for y in predicted)] for x in np.unique(target))

    """
    Metoda care calculeaza recall-ul pentru o clasificare multi class
    @params: predicted - the classes predicted by the model
             target - the actual classes
    @return: integer
    """
    @classmethod
    def MultiRecall(cls, predicted, target):
        return dict([x, sum([y == z for y, z in zip(predicted, target) if y == x]) / sum(y == x for y in target)] for x in np.unique(target))
    
    """
    Metoda care calculeaza acuratetea, precizia si recall-ul pentru o clasificare cu mai mult de 2 output-uri posibile
    @params: predicted - the classes predicted by the model
             target - the actual classes
    @return: un dictionar care cuprinde metricile
    """
    @classmethod
    def MultiMetrics(cls, predicted, target):
        return {'precision': ClassificationMetrics.MultiPrecision(predicted, target), 'recall':  ClassificationMetrics.MultiRecall(predicted, target)}

    """
    Metoda care calculeaza cross-entropy loss pentru o clasificare binara
    @params: matrix - a matrix of dimension noSamples x 2, each row having probabilities that sum up to 1 
             target - the actual classes
    @return: integer
    """
    @classmethod
    def BinaryCrossEntropy(cls, matrix, target):
        return - (1 / len(matrix)) * sum([(t * math.log2(row[0]) + (1 - t) * math.log2(row[1])) for row, t in zip(matrix, target)])

    """
    Metoda care calculeaza cross-entropy loss pentru o clasificare multi-class
    @params: matrix - a matrix of dimension noSamples x noClasses, each row having probabilities that sum up to 1 
             target - the actual classes
    @return: integer
    """
    @classmethod
    def CrossEntropy(cls, matrix, target):
        return - (1 / len(matrix)) * sum([sum([t * math.log2(r) for t, r in zip(tar, row)]) for row, tar in zip(matrix, target)])

    """
    Metoda care calculeaza cross-entropy loss pentru o clasificare multi-label
    @params: matrix - a matrix of dimension noSamples x noClasses, each row having probabilities of having the correspoding label
             target - the actual classes
    @return: java
    """
    @classmethod
    def MultiLabelLoss(cls, matrix, target):
        return - (1 / len(matrix)) * sum([sum([t * math.log2(r) + (1 - t) * math.log2(1 - r) for t, r in zip(tar, row)]) for row, tar in zip(matrix, target)])

In [4]:
def readData(filename):
    df = pd.read_csv(filename)

    predicted = [] 
    target = [] 

    predictedColumns = [x for x in df.columns if 'predicted' in x.lower()]
    targetColumns = [x for x in df.columns if 'predicted' not in x.lower()]

    for _, row in df.iterrows():
        predicted.append([row[x] for x in predictedColumns])
        target.append([row[x] for x in targetColumns])
    
    if len(predictedColumns) == 1:
        return [np.squeeze(predicted), np.squeeze(target)]
    return [np.array(predicted), np.array(target)]

In [5]:
[predicted, target]  = readData('sports.csv')
RegressionMetrics.MultiMAE(predicted, target)

8.15

In [6]:
predicted = [1, 0, 1, 0, 0]
target = [1, 0, 0, 1, 0]
ClassificationMetrics.BinaryMetrics(predicted, target, 1, 0)

{'accuracy': 0.6,
 'precision positive': 0.5,
 'recall positive': 0.5,
 'precision negative': 0.6666666666666666,
 'recall negative': 0.6666666666666666}

In [7]:
[predicted, target]  = readData('flowers.csv')
print(ClassificationMetrics.MultiPrecision(predicted, target))
print(ClassificationMetrics.MultiRecall(predicted, target))

{'Daisy': 0.36363636363636365, 'Rose': 0.25, 'Tulip': 0.2857142857142857}
{'Daisy': 0.36363636363636365, 'Rose': 0.25, 'Tulip': 0.2857142857142857}


In [8]:
matrix = [[0.8, 0.2], [0.25, 0.75]]
target = [1, 0]
print(ClassificationMetrics.BinaryCrossEntropy(matrix, target))

0.36848279708310305


In [9]:
matrix = [[0.8, 0.19, 0.01], [0.2, 0.75, 0.05], [0.23, 0.17, 0.6]]
target = [[1, 0, 0], [0, 1, 0], [0, 0, 1]]
print(ClassificationMetrics.CrossEntropy(matrix, target))

0.49131039611080407


In [10]:
matrix = [[0.9, 0.19, 0.95], [0.2, 0.75, 0.85], [0.8, 0.17, 0.9]]
target = [[1, 0, 1], [0, 1, 1], [1, 1, 1]]
print(ClassificationMetrics.MultiLabelLoss(matrix, target))

1.510588415479651
